In [1]:
%cd ../../

%load_ext autoreload
%autoreload 2

/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract


In [4]:
import torch
from PIL import Image
import numpy as np
import cv2
from torchvision import transforms

from src.handlers import utils

In [5]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010],
        ),
    ]
)


In [6]:
path = "/Users/macos/Downloads/file_example_MOV_1280_1_4MB.mov"

cap = cv2.VideoCapture(path)

frames = []
n = 0
while cap.isOpened():
    if n % 10 == 0:
        _, frame = cap.read()
        if frame is None:
            break

        img = Image.fromarray(frame)
        img_trans = transform(img)[None, :].numpy()

        frames.append(img_trans)
        # feat = utils.extract_feat(frame)

    n = (n + 1) % 10

cap.release()

In [11]:
a = np.concatenate(frames, axis=0)
a.shape

(901, 3, 224, 224)

In [12]:
ort_inps = {utils.ort_session.get_inputs()[0].name: a}
ort_outs = utils.ort_session.run(None, ort_inps)


In [17]:
np.mean(ort_outs[0], axis=0).shape

(2048,)

In [14]:
path1 = "/Users/macos/.config/iterm2/naomi_pic/IMG_20210604_000806_995.jpg"
path2 = "/Users/macos/.config/iterm2/naomi_pic/IMG_20210604_000808_719.jpg"

with open(path1, 'rb') as fp1, open(path2, 'rb') as fp2:
    file1, file2 = fp1.read(), fp2.read() 

torch.nn.functional.cosine_sim ilarity(
    torch.tensor(utils.extract_feat(file1)),
    torch.tensor(utils.extract_feat(file2))
)

tensor([1.0000])

In [15]:
a = [
    np.random.rand(5)
    for _ in range(4)
]

a

[array([0.58516993, 0.97659382, 0.7380996 , 0.12003575, 0.87701033]),
 array([0.89098443, 0.6549494 , 0.52801826, 0.50884251, 0.52136998]),
 array([0.76238492, 0.10880999, 0.10022516, 0.08172431, 0.09084703]),
 array([0.17545779, 0.45657156, 0.91978557, 0.59043415, 0.76246391])]

In [17]:
np.mean(a, axis=-1)

array([0.65938189, 0.62083292, 0.22879828, 0.5809426 ])